# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [7]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [8]:
directory_read_tool = DirectoryReadTool(directory='./instructions') ## Read Local intructions.
## Directory contains series of Intructions on "How to handle companies with different sizes"
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

Instructions Directory contians 3 files :
- Customer Engagement Guide for Small Businesses
- Strategy and Inovation Framework for Enterprise Solution
- Personalized Outreach Intructions for Tech Startups 

Agent can use this to ans different company sizes

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [9]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In Run Function , You can include anything . It can be external API Call or anyother logic

In [10]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
        ### Description will tell agent . When to use this tool
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [11]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [12]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [13]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [14]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather information on DeepLearningAI to create a comprehensive report for our sales team.

Action: Read a file's content
Action Input: {"file_path": "./instructions/DeepLearningAI_company_background.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: './instructions/DeepLearningAI_company_background.txt'.
 Tool Read a file's conte

Thought: I now know the final answer

Final Answer: 

**DeepLearningAI Company Profile:**

DeepLearningAI is a company founded in 2017 by Andrew Ng, a global leader in AI. The company explores the frontier of AI and aims to make world-class AI education accessible to people around the globe. DeepLearningAI offers online courses and specializations in AI and machine learning.

**Key Personnel:**

- Founder & CEO: Andrew Ng
- Managing General Partner: AI Fund LP
- Founder: DeepLearning.AI LLC and Landing AI, Inc.

**Recent Milestones:**

- The company recently reached the milestone of the 256th issue of The Batch, reflecting on how AI has changed over the years and the impact on society.
- DeepLearningAI offers expert-led educational experiences on Coursera, providing AI practitioners and non-technical professionals with essential tools.
- The Machine Learning Specialization has been newly rebuilt and expanded into 3 courses to teach foundational AI concepts.

**Identified Needs:**

Deep

Final Answer:

Email Draft to Andrew Ng, CEO of DeepLearningAI:

Subject: Enhancing AI Education with CrewAI's Innovative Solutions

Dear Andrew Ng,

I hope this email finds you well. I am reaching out to you on behalf of CrewAI, a cutting-edge technology company that specializes in providing advanced tools and resources to enhance online learning platforms.

I recently came across the remarkable achievements of DeepLearningAI, particularly the milestone of the 256th issue of The Batch, which reflects on the evolution of AI and its profound impact on society. Your commitment to making world-class AI education accessible to a global audience is truly inspiring.

At CrewAI, we share a similar passion for advancing AI education and empowering individuals to develop essential skills in the field. Our solutions are designed to provide personalized learning experiences, real-time feedback, and valuable insights to optimize AI skill development.

I believe that by integrating CrewAI's innovat

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Email Draft to Andrew Ng, CEO of DeepLearningAI:

Subject: Enhancing AI Education with CrewAI's Innovative Solutions

Dear Andrew Ng,

I hope this email finds you well. I am reaching out to you on behalf of CrewAI, a cutting-edge technology company that specializes in providing advanced tools and resources to enhance online learning platforms.

I recently came across the remarkable achievements of DeepLearningAI, particularly the milestone of the 256th issue of The Batch, which reflects on the evolution of AI and its profound impact on society. Your commitment to making world-class AI education accessible to a global audience is truly inspiring.

At CrewAI, we share a similar passion for advancing AI education and empowering individuals to develop essential skills in the field. Our solutions are designed to provide personalized learning experiences, real-time feedback, and valuable insights to optimize AI skill development.

I believe that by integrating CrewAI's innovative technology into DeepLearningAI's online learning platform, we can further enhance the educational experiences of your students and bridge the gap between theoretical knowledge and practical application. Our tools align perfectly with DeepLearningAI's mission and values, offering a unique opportunity to elevate the impact of your educational offerings.

I would love the opportunity to offer you a personalized demo of CrewAI's solutions tailored to DeepLearningAI's specific needs and goals. Additionally, I can provide you with case studies showcasing successful implementations of CrewAI in similar online learning platforms.

I am excited about the possibility of collaborating with DeepLearningAI to educate your audience on the benefits of AI-powered learning tools through joint webinars or events. Together, we can contribute to the advancement of AI education worldwide and make a meaningful impact on the future of the industry.

I look forward to the opportunity to discuss this further with you and explore how CrewAI can support DeepLearningAI in achieving its goals. Please let me know a convenient time for a meeting or demo presentation.

Thank you for considering this partnership opportunity. I am eager to hear your thoughts and explore how we can work together to enhance AI education for individuals around the globe.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---
Email Draft 2:

Subject: Elevating AI Education with CrewAI's Innovative Solutions

Dear Andrew Ng,

I hope this message finds you well. As the Founder and CEO of DeepLearningAI, you have been at the forefront of revolutionizing AI education and making it accessible to a global audience.

I was impressed by the recent milestones achieved by DeepLearningAI, especially the rebuilding and expansion of the Machine Learning Specialization into 3 courses. Your dedication to providing high-quality AI education and addressing the growing demand for AI skills is truly commendable.

At CrewAI, we are dedicated to enhancing online learning platforms with our advanced tools and resources. Our solutions offer personalized learning experiences, real-time feedback, and valuable insights to optimize AI skill development.

By integrating CrewAI's innovative technology into DeepLearningAI's online learning platform, we can further elevate the educational experiences of your students and empower them to excel in the field of AI. Our tools are designed to align seamlessly with DeepLearningAI's mission and goals, creating a unique opportunity for collaboration.

I would be delighted to offer you a tailored demo of CrewAI's solutions to showcase how they can support DeepLearningAI's specific needs and objectives. Additionally, I can provide you with case studies demonstrating successful implementations of CrewAI in similar online learning platforms.

I am excited about the prospect of collaborating with DeepLearningAI to educate your audience on the benefits of AI-powered learning tools through joint webinars or events. Together, we can contribute to the advancement of AI education worldwide and make a lasting impact on the industry.

I look forward to the opportunity to discuss this further with you and explore how CrewAI can help DeepLearningAI achieve its educational goals. Please let me know a convenient time for a meeting or demo presentation.

Thank you for considering this partnership opportunity. I am eager to hear your feedback and discuss how we can work together to enhance AI education for individuals globally.

Best regards,

[Your Name]
Lead Sales Representative
CrewAI